# Rockets notebook - more efficient and clean

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import time

# model imports
from sklearn import model_selection
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report, log_loss
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

sns.set_style('white')
import random

%matplotlib inline

random.seed(12345)
seed = 12345  # for seeding individually

pd.options.display.max_columns = 100  # max columns to display

In [23]:
# models to enumerate
model_list = [ExtraTreesClassifier(),
RandomForestClassifier(),
GradientBoostingClassifier(),
XGBClassifier()]

In [24]:
def get_classifier_name(classifier):
    return str(classifier).split('(')[0]

# so that: 
# get_classifier_name(model_list[0])  -> 'DecisionTreeClassifier'


In [25]:
def process_data(train_data, has_labels=True):
    
    if has_labels:
        target = train_data.loc[:,'class']

    # parsing features and turnng them into vectors:
    qq = list(range(1, 211, 7))
    t_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(2, 211, 7))
    x_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(3, 211, 7))
    y_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(4, 211, 7))
    z_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(5, 211, 7))
    vx_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(6, 211, 7))
    vy_mat = train_data.iloc[:,qq].as_matrix()
    qq = list(range(7, 211, 7))
    vz_mat = train_data.iloc[:,qq].as_matrix()

    # 3 matrices:
    ax_mat = np.gradient(vx_mat,0.5,axis=1)
    ay_mat = np.gradient(vy_mat,0.5,axis=1)
    az_mat = np.gradient(vz_mat,0.5,axis=1)
    
    # features:
    v_total = np.sqrt(np.power(vx_mat,2)+np.power(vy_mat,2)+np.power(vz_mat,2))
    a_total = np.sqrt(np.power(ax_mat,2)+np.power(ay_mat,2)+np.power(az_mat,2))
    vx_mean = np.nanmean(vx_mat,axis=1)
    vz_mean = np.nanmean(vz_mat,axis=1)
    v_mean = np.nanmean(v_total,axis=1)
    ax_mean = np.nanmean(ax_mat,axis=1)
    az_mean = np.nanmean(az_mat,axis=1)
    a_total_mean = np.nanmean(a_total,axis=1)
    z_mean = np.nanmean(z_mat,axis=1)
    vx_std = np.nanstd(vx_mat,axis=1)
    vy_std = np.nanstd(vy_mat,axis=1)
    vz_std = np.nanstd(vz_mat,axis=1)
    v_std = np.nanstd(v_total,axis=1)
    ax_std = np.nanstd(ax_mat,axis=1)
    ay_std = np.nanstd(ay_mat,axis=1)
    az_std = np.nanstd(az_mat,axis=1)
    a_total_std = np.nanstd(a_total,axis=1)
    y_std = np.nanstd(y_mat,axis=1)
    N_data = len(train_data)
    
    notnan = ~np.isnan(x_mat)
    p=np.zeros((N_data, 3))
    roots = np.zeros((N_data, 2))
    for i in range(N_data):
        p[i,:] = np.polyfit(x_mat[i,notnan[i,:]], z_mat[i,notnan[i,:]],2)
        roots[i]=np.roots(p[i])

    poly_range = np.abs(roots[:,1]-roots[:,0])

    poly_c0 = p[:,0]
    poly_c1 = p[:,1]
    poly_c2 = p[:,2]

    poly_maxz = poly_c2-np.power(poly_c1,2)/float(4)/poly_c0
    
    # the dict that will later become the output data frame
    return_dict = {'z_mean': z_mean, 'vx_mean': vx_mean, 'vz_mean': vz_mean, 'v_mean': v_mean,
                   'ax_mean':ax_mean,'az_mean':az_mean,'a_total_mean':a_total_mean,
                   'vx_std': vx_std, 'vz_std': vz_std, 'v_std': v_std, 'ax_std':ax_std,
                   'az_std':az_std,'a_total_std':a_total_std, 'poly_c0':poly_c0,
                   'poly_c1':poly_c1,'poly_c2':poly_c2, 'poly_range':poly_range,'poly_maxz':poly_maxz}
    
    # if it is the training set, add the labels
    #     if has_labels: 
    #         return_dict['class'] = target
    
    
    new_feat = pd.DataFrame(return_dict)
    
    # get first 5 seconds
    qq = list(range(1,71))
    X_raw = train_data.iloc[:,qq]
    
    #     X_new = new_feat.drop('class',axis=1)

    # X = X_raw.join(X_new)  # Not using RAW at all
    X = new_feat
    if has_labels:
        y = target
        
        # then split to train test
        X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=1)
        
        return X_train, X_val, y_train, y_val
    else:
        return X
        



In [26]:
# DATA LOADING - change work_folder to your folder
work_folder = r'E:\DataHack_2017'  # 
folder_join = os.path.join
test_sample = folder_join(work_folder, 'test.csv')
train_sample = folder_join(work_folder, 'train.csv')

In [27]:
# saving two least common classes for later, for 'other' classification
data_with23_16 = pd.read_csv(train_sample)
data_no23_16 = data_with23_16[~data_with23_16['class'].isin([23,16])]  # removed 2 classes as other
train_data, test_data = train_test_split(data_no23_16,test_size=0.2,random_state=123)  

# TODO: remember 23,16

In [33]:
def model_enumeration(X, y, model_list, scoring_list):
    print("Running model enumeration:")
    st_st_time = time.time()  # timing
    
    for model in model_list:
        name = get_classifier_name(model)  # the name of the model 
        st_time = time.time()
        print(f"Working on model {name}")
        kfold = model_selection.KFold(n_splits=10, random_state=seed)
        for scoring_type in scoring_list:
            result = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring_type, n_jobs=-1)
            print(f"{name}: {scoring_type}: {result.mean():2.2f}, ({result.std():2.2f})")
        print(f"{name} took {time.time() - st_time:2.2f}")

    print(f"The run took {time.time() - st_st_time:2.2f}")

In [29]:
# if that's the training set:
X_train, X_val, y_train, y_val = process_data(train_data)

# if that's the real set
# X_test = process_data(test_data, has_labels=False)

In [34]:
# run model enumeration
scoring_list = ['accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted']
model_enumeration(X_train, y_train, model_list, scoring_list)

Running model enumeration:
Working on model DecisionTreeClassifier
DecisionTreeClassifier: accuracy: 0.49, (0.01)
DecisionTreeClassifier: f1_weighted: 0.49, (0.01)
DecisionTreeClassifier: precision_weighted: 0.49, (0.01)
DecisionTreeClassifier: recall_weighted: 0.49, (0.01)
DecisionTreeClassifier took 60.20
Working on model ExtraTreesClassifier
ExtraTreesClassifier: accuracy: 0.55, (0.01)
ExtraTreesClassifier: f1_weighted: 0.54, (0.01)
ExtraTreesClassifier: precision_weighted: 0.55, (0.01)
ExtraTreesClassifier: recall_weighted: 0.55, (0.01)
ExtraTreesClassifier took 62.54
Working on model RandomForestClassifier
RandomForestClassifier: accuracy: 0.55, (0.01)
RandomForestClassifier: f1_weighted: 0.55, (0.01)
RandomForestClassifier: precision_weighted: 0.55, (0.02)
RandomForestClassifier: recall_weighted: 0.55, (0.01)
RandomForestClassifier took 69.47
Working on model GradientBoostingClassifier
GradientBoostingClassifier: accuracy: 0.54, (0.01)
GradientBoostingClassifier: f1_weighted: 0.5

KeyboardInterrupt: 

In [41]:
# Parameter optimization
# TODO: grid
# on randomForest and ExtraTrees
param_grid = {'n_estimators': [5, 10, 15, 20, 30, 50, 80, 100],
              'max_depth': [2, 5, 7, 8, 9, 12, 14],
             }

st_time = time.time()
# for clf in (RandomForestClassifier(n_jobs=-1)
clf = RandomForestClassifier(n_jobs=-1)
print(f"Working on model {clf}")
grid_clf = model_selection.GridSearchCV(clf, param_grid, scoring='accuracy', cv=10)
# grid_clf = model_XGBClassifier(objective='multi:softprob',learning_rate=0.2,
#                     subsample=0.7,
#                     colsample_bytree=0.9,
#                     colsample_bylevel=0.7,
#                     max_depth=8,
#                     nthread=4,
#                     n_estimators=100,
#                     seed=1234))selection.GridSearchCV(clf, param_grid, scoring='f1_weighted', cv=10)
grid_clf.fit(X_train, y_train)
# print(f"{clf}: {scoring_type}: {result.mean():2.2f}, ({result.std():2.2f})")


Working on model RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15, 20, 30, 50, 80, 100], 'max_depth': [2, 5, 7, 8, 9, 12, 14]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [45]:
# for random tree:
print(f"best params: {grid_clf.best_params_}")
print(f"Best score: {grid_clf.best_score_:2.2f}")

best params: {'max_depth': 14, 'n_estimators': 100}
Best score: 0.61


In [ ]:
param_grid_xgboost = {'learning_rate': [0.2/3, 0.2, 0.6, 0.9, 1.5], 
                      'max_depth': list(range(5,16)),
                      'n_estimators': [50, 100, 150, 200]}
st_time = time.time()
# clf = RandomForestClassifier(n_jobs=-1)
name = get_classifier_name(model)  # the name of the model
print(f"Working on model {name}")
# grid_clf = model_selection.GridSearchCV(clf, param_grid, scoring='accuracy', cv=10)
xg_clf = XGBClassifier(objective='multi:softprob',
                    subsample=0.7,
                    colsample_bytree=0.9,
                    colsample_bylevel=0.7,
                    nthread=4,
                    seed=seed)
grid_xg_clf = model_selection.GridSearchCV(xg_clf, param_grid_xgboost, scoring='f1_weighted', cv=10)
grid_xg_clf.fit(X_train, y_train)

Working on model RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [ ]:
def run_PCA_on_data(data):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(data)
    print(pca.explained_variance_ratio_)
    return X_pca

In [ ]:
def run_tSNE_on_data(data, init='random'):
    X_embedded = TSNE(n_components=2, init=init).fit_transform(X)
    return X_embedded


In [ ]:
# Also, run PCA
X_pca = run_PCA_on_data(X_train)

# no PCA
X_tSNE = run_tSNE_on_data(X_train)
# start with PCA
X_tSNE = run_tSNE_on_data(X_train, init='pca')

# tODO: run in classifier

In [ ]:
def plot_PCA(X_pca):
    """Plot PCA figure."""
    plt.figure(figsize=(15, 7))
    p1 = [x[0] for x in X_pca]
    p2 = [x[1] for x in X_pca]

    colors = [int(i % 500) for i in y_train]
    plt.scatter(p1, p2, c=colors, s=3)
    sns.despine()

In [ ]:
# gbc = XGBClassifier(objective='multi:softprob',
#                     learning_rate=0.2,
#                     subsample=0.7,
#                     colsample_bytree=0.9,
#                     colsample_bylevel=0.7,
#                     max_depth=8,
#                     nthread=4,
#                     n_estimators=100,
#                     seed=1234)